In [10]:
spark.stop()

In [6]:
import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
import time
from pyrecdp.data_processor import *
from pyrecdp.encoder import *
from pyrecdp.utils import *

spark = SparkSession.builder.master('yarn')\
        .appName("Recsys2021_data_process")\
        .getOrCreate()
current_path = "/recsys2021_0608_processed/"
path_prefix = "hdfs://"
dicts_folder = "recsys_dicts/"

In [7]:
from pyspark.ml.feature import *
train_data_path= "/recsys2021_0608_processed/tweet_text_processed_fixed"
train = spark.read.parquet("%s/" % (train_data_path))
df = train.select('original_engaged_with_user_id', 'engaged_with_user_follower_count')\
          .groupby('original_engaged_with_user_id')\
          .agg(f.max(f.col('engaged_with_user_follower_count')))\
          .withColumnRenamed('max(engaged_with_user_follower_count)', 'engaged_with_user_follower_count')
print("after agg, total number of engaged_with_user_id is %d" % df.count())
df.show()
df = df.select('engaged_with_user_follower_count').distinct()
qd = QuantileDiscretizer(numBuckets=100, inputCol="engaged_with_user_follower_count", outputCol='follower_quantile')
result  = qd.fit(df).transform(df)
result.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_engaged_with_user_id_follower_quantile')
#qdmodel = qd.fit(df)
#display(qdmodel.getSplits())

Py4JJavaError: An error occurred while calling o267.count.
: org.apache.spark.SparkException: Job 1 cancelled because killed via the Web UI
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2253)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2149)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleStageCancellation$1(DAGScheduler.scala:2138)
	at scala.runtime.java8.JFunction1$mcVI$sp.apply(JFunction1$mcVI$sp.java:23)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:246)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:2131)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2396)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(